In [1]:
import os
from selenium import webdriver

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

#driver = webdriver.Chrome(chromedriver)
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

import sys

import json

In [2]:
driver = webdriver.Chrome(chromedriver)

In [3]:
#head = web link
driver.get(head)
page = driver.page_source
soup = BeautifulSoup(page,'html.parser')

In [6]:
class Scraper_web(object):
    def __init__(self):
        pass
    
    def Input_Search(self, my_driver, what_content, where_content):
        #my_driver, chrome web driver, alreay open the current page
        #what_content, the input to "what" field, type: string
        #where_content, the input to 'where" field type: string
        self.driver = my_driver
        
        #input what_content and where_content
        search_title = self.driver.find_element_by_xpath("//input[@id = 'input-q']")
        search_title.clear()
        search_title.send_keys(what_content)
        
        search_location = self.driver.find_element_by_xpath("//input[@id = 'input-l']")
        search_location.clear()
        search_location.send_keys(where_content)
        
        find_resume_button = self.driver.find_element_by_xpath("//button[text() = 'Find resumes']")
        find_resume_button.click()
        self.pre_address = self.driver.current_url
        
    def _Next_page(self):
        try:
            
            next_button = self.driver.find_element_by_class_name('rezemp-pagination-nextbutton')
            #print(next_button.text)    #for test
            #next_button.click()
            #print(self.driver.current_url)
            #print('*******')
            has_error_message = self.driver.find_element_by_xpath("//span[@class = 'icl-Alert-headline']")
            if (has_error_message.text == 'Error'):
                print('page has error information')
                return False
            else:
                self.driver.execute_script("arguments[0].click();", next_button)
                return self.driver.page_source
        except:
            print('No next element')
            return False
        
        
    def _Current_page(self, add_address=''):
        #add_page = '&start=' + str(50)
        #pre_address = driver.current_url
        current_address = self.pre_address + add_address
        #print(current_address)
        if (add_address !=''):
            self.driver.get(current_address)
            
        return self.driver.page_source
        
    def Find_Links(self, current_page):
        #current_page is the current page source code
        #page = driver.page_source
        soup = BeautifulSoup(current_page,'html.parser')
        find_resume = soup.find_all("div", class_="rezemp-ResumeSearchCard")
        
        number_resume = len(find_resume)
        resume_link = []
        for i in range(number_resume):
            link = find_resume[i].find('a')['href']
            resume_link.append(link)
        return resume_link
        
    def Open_link(self, input_links, input_array):
        #input_array has resume information added
        first_part_link = 'https://resumes.indeed.com'
        number_resume = len(input_links)
        for i in range(number_resume):
            current_link = first_part_link + input_links[i]
            self.driver.execute_script("window.open('');")
            #time.sleep(3)
            self.driver.switch_to.window(driver.window_handles[1])
            self.driver.get(current_link)
            #time.sleep(3)
            open_page = self.driver.page_source
            
            #start_time = time.time()
            #*****************
            #extract page information
            self.Extract_page_info(i, open_page, input_array)
            #end_time = time.time()
            #print('time spent is: ', end_time - start_time)
            #******************
            
            self.driver.close()
            self.driver.switch_to.window(driver.window_handles[0])
#Work Experience
#Education
#Skills
#Additional Information
    def _Create_list(self, page_content, category_type):
        #input page_content, soup object
        #category_type  string object
        #return new_list  list type
        category_list = ['Work Experience', 'Education',
                            'Skills', 'Additional Information']
        new_list =[]
        if category_type == category_list[0]:
            #Work Experience
           
            work_experience_list = page_content.find_all('div', class_ = 'rezemp-WorkExperience')
            number_work = len(work_experience_list)
            for i in range(number_work):
                new_dict = {}
                work_title = work_experience_list[i].find('div', 
                                                          class_ = 'rezemp-u-h4')
                new_dict['title'] = work_title.get_text()
                work_subtitle = work_experience_list[i].find('div', 
                                                             class_ = 'rezemp-WorkExperience-subtitle')
                work_company = work_subtitle.find('div')
                new_dict['company'] = work_company.get_text()
                work_time = work_company.find_next_sibling('div')
                if work_time ==None:
                    new_dict['dates'] = 'NA'
                else:
                    new_dict['dates'] = work_time.get_text()
                job_description = work_subtitle.find_next_sibling('div')
                if job_description ==None:
                    new_dict['description'] = 'NA'
                else:
                    new_dict['description'] = job_description.get_text().\
                                          replace(u'\xa0', '').replace('\t', '').replace('•',' ')
                new_list.append(new_dict)
        elif category_type == category_list[1]:
            #Education
            #new_dict = {}
            education_content = page_content.find('div', 
                                                  class_ = 'rezemp-ResumeDisplaySection-content')
            degree_list = education_content.find_all('span', 
                                                     class_ = 'rezemp-ResumeDisplay-itemTitle')
            school_list = education_content.find_all('div', 
                                                         class_ = 'rezemp-ResumeDisplay-university')
            number_school = len(degree_list)
            for i in range(number_school):
                new_dict = {}
                new_dict['title'] = degree_list[i].get_text()
                school_name = school_list[i].find('span').get_text()
                new_dict['school'] = school_name
                school_location = school_list[i].find('span', 
                                                      class_ = 'rezemp-ResumeDisplay-universityLocation').get_text()
                new_dict['location'] = school_location
                school_dates = school_list[i].find('span', 
                                                   class_ = 'rezemp-ResumeDisplay-date').get_text()
                new_dict['dates'] = school_dates
                
                new_list.append(new_dict)
        elif category_type == category_list[2]:
            #Skills
            #new_dict = {}
            skills = page_content.find('div', 
                                       class_ = 'rezemp-ResumeDisplaySection-content')
            skills_string = skills.get_text()
            new_list.append(skills_string)
        else:
            #Additional Information
            new_dict = {}
            additional_info = page_content.find('div', 
                                                class_ = 'rezemp-ResumeDisplaySection-content')
            additional_info_string= additional_info.get_text()
            new_list.append(additional_info_string)
      
        return new_list
 



        
            
            
    def Extract_page_info(self, person_index, current_page, input_array):
        #current_page is source code of the webpage
        #current_page = driver.page_source
        #append the extracted information to input_array
        soup = BeautifulSoup(current_page,'html.parser')
        page_content = soup.find_all('div', class_ = 'rezemp-ResumeDisplaySection')
        number_content = len(page_content)
        
        new_dict = {}
        new_dict['index'] = person_index
        summary_part = soup.find('div', class_ = 'rezemp-ResumeDisplay-body').find('div')
        #***********************
        title = summary_part.find('div', class_ = 'rezemp-u-h2')
        new_dict['title'] = title.get_text()
        title_sub = summary_part.find('div', class_ = 'rezemp-u-h4')
        new_dict['subtitle'] = title_sub.get_text()
        title_loc = title_sub.find_next_sibling('div')
        new_dict['location'] = title_loc.get_text()
        #**************************
        #loop through page_content to find out which
        #it belongs to the category for output
        category_set = set(['Work Experience', 'Education',
                            'Skills', 'Additional Information'])
        #initialization
        new_dict['work_experience'] = 'NA'
        new_dict['education'] = 'NA'
        new_dict['skills'] = 'NA'
        new_dict['additional_information'] = 'NA'
        
        for i in range(number_content):
            category_type = page_content[i].find('span', 
                                                 class_ = 'rezemp-ResumeDisplaySection-header').get_text()
            if category_type in category_set:
                if (category_type == 'Work Experience'):
                    new_dict['work_experience'] = self._Create_list(page_content[i], category_type)
                elif (category_type == 'Education'):
                    new_dict['education'] = self._Create_list(page_content[i], category_type)
                elif (category_type == 'Skills'):
                    new_dict['skills'] = self._Create_list(page_content[i], category_type)
                else:
                    new_dict['additional_information'] = self._Create_list(page_content[i], category_type)
                    
        input_array.append(new_dict)
            

In [7]:
test_scraper = Scraper_web()

In [8]:
#1. California
#2. Washington
#3. Oregon
#4. Alabama
#5. Arizona
#6. Arkansas
#7. Colorado
#8. Connecticut
#9. Delaware
#10. Florida
#11. Georgia
#12. Hawaii
#13. Idaho
#14. Illinois
#15. Indiana
#16. Iowa
#17. Kansas
#18. Kentucky
#19. Louisiana
#20. Maine
#21. Maryland
#22. Massachusetts
#23. Michigan
#24. Minnesota
#25. Mississippi
#26. Missouri
#27. Nebraska
#28. Nevada
#29. New Hampshire
#30. New Jersey
#31. New Mexico
#32. New York
#33. North Carolina
#34. North Dakota
#35. Ohio
#36. Oklahoma
#37. Pennsylvania
#38. Rhode Island
#39. South Carolina
#40. South Dakota
#41. Tennessee
#42. Texas
#43. Utah
#44. Vermont
#45. Virginia
#46. Wisconsin
#The other states do not have resume with this title
title = ['Data Scientist', 'Data Analyst','Machine Learning Engineer', 
         'Data Engineer', 'Software Engineer', 'Engineer', 'Web Developer',
         'Statistician', 'Electrical Engineer', 'Financial Analyst',
         'Process Engineer', 'Computer Engineer']
state_name = ['California', 'Washington', 'Oregon', 'Alabama',  'Arizona', 
              'Arkansas', 'Colorado','Connecticut', 'Delaware',
              'Florida','Georgia', 'Hawaii', 'Idaho', 'Illinois',
              'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
              'Maine', 'Maryland', 'Massachusetts','Michigan', 'Minnesota',
              'Mississippi', 'Missouri', 'Nebraska', 'Nevada', 'New Hampshire',
              'New Jersey', 'New Mexico', 'New York', 'North Carolina','North Dakota',
              'Ohio', 'Oklahoma', 'Pennsylvania', 'Rhode Island', 'South Carolina',
              'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
              'Virginia', 'Wisconsin']
            

#about 5 seconds

In [ ]:
start_time = time.time()
what_content = title[0]
where_content = state_name[2]
test_scraper.Input_Search(driver, what_content, where_content)
end_time = time.time()
print('time spent is: ', end_time - start_time)

input_array =[]
has_next = True
start_time = time.time()
page = test_scraper._Current_page()
resume_link = test_scraper.Find_Links(page)
test_scraper.Open_link(resume_link, input_array)
i=1
while has_next:
    page = test_scraper._Next_page()
    #time.sleep(5)
    sys.stdout.write(" P"+str(i))
    i +=1
    has_next = page
    if has_next:
        resume_link = test_scraper.Find_Links(page)
        test_scraper.Open_link(resume_link, input_array)
end_time = time.time()
print('Total time spent is: ', end_time - start_time)

print(len(input_array))

test_scraper.driver.close()

save_directory ='./scraper_data/'
file_index = 3
name_base = 'file'
file_name = save_directory + name_base + str(file_index)+'.json'

with open(file_name, 'w') as f:
    json.dump(input_array, f)

driver.close()

In [19]:
#j=32 has some issue
#<50 for data scientist    0
#[50,100) for data analyst  1
#[150,200) machine learning engineer  2
#[100, 150) Data Engineer 3
#[200, 250)  Software Engineer 4
#[250,300] Engineer   5
save_directory ='./scraper_data/'
file_index_base = 250
title_number = 5
what_content = title[title_number]
name_base = 'file'
number_states = len(state_name)
j=43   
print('Title in search is: ', what_content)
while j<number_states:
    driver = webdriver.Chrome(chromedriver)
    driver.get(new_link)
    
    test_scraper = Scraper_indeed()
    #what_content = title[title_number]
    where_content = state_name[j]
    test_scraper.Input_Search(driver, what_content, where_content)
    time.sleep(25)
    input_array =[]
    has_next = True
    start_time = time.time()
    page = test_scraper._Current_page()
    resume_link = test_scraper.Find_Links(page)
    test_scraper.Open_link(resume_link, input_array)
    i=1
    while has_next:
        page = test_scraper._Next_page()
        has_next = page
        if has_next:
            resume_link = test_scraper.Find_Links(page)
            test_scraper.Open_link(resume_link, input_array)
    end_time = time.time()
    print('j is: ', j, 'state is:', where_content, ' Total time spent is: ', end_time - start_time)
    test_scraper.driver.close()
    
    file_name = save_directory + name_base + str(j+file_index_base)+'.json'
    with open(file_name, 'w') as f:
        json.dump(input_array, f)
    j +=1

Title in search is:  Engineer
No next element
j is:  43 state is: Vermont  Total time spent is:  253.47414088249207
page has error information
j is:  44 state is: Virginia  Total time spent is:  537.6544470787048


AttributeError: 'NoneType' object has no attribute 'find'

In [20]:
test_scraper.driver.close()
test_scraper.driver.switch_to.window(driver.window_handles[0])
test_scraper.driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.3 x86_64)


In [21]:
test_scraper.driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.3 x86_64)


In [ ]:
driver.close()